# Lesson 1 - Calibration Overview

# Dynamically Dimensioned Search:
We need some general content on the DDS ... 


# Programs & Workflow:
Calibration workflow is run through a series of six individual python scripts, typically run from the command line (terminal). The python scripts must be run in order, and only once the previous script has completed. Brief descriptions of each script are provided below and more detailed explanation would be provided in the upcoming lessons. 

![WorkFlow_Image_Placeholder](imagename.png "Image Placeholder for now")

*initDB.py:* 

This program is used one time to initialize the calibration database and associated tables used during the experiment. Completing this step will create an empty database with all the relevant tbales that will be filled with information provided during the calibration procedure. More information will be provided in Lesson 2. 


*inputDomainMeta.py:* 

This program reads in a CSV file you will need to fill out that describes modeling domains to be used for calibration. This information is entered into the database for later workflow use. More description on the CSV will occur during the setup section (Lesson 2).

*jobInit.py:* 

This program is run to establish a calibration ‘experiment’. The program reads a config file 
(explained in depth below) and sets up the necessary run directories, paths to necessary files, and inputs associated metadata into the database. Upon successful completion, the program will return a unique job ID value which you will use in subsequent programs to run the calibration.

*spinOrchestrator.py*

 This is the first program that is run to initialize the calibration experiment. The only mandatory argument to this program is the unique job ID for the calibration experiment. The main purpose of this program is to run the NWM/WRF-Hydro spinup for all domains being calibrated. This program needs to successfully complete before moving onto the next step.


*calibOrchestrator.py:* 

This is the second program that is run in the calibration workflow. As with spinup.py, the only mandatory argument is the job ID value. This program runs the main workflow to adjust parameter values, execute interim model simulations, evaluate model output against observations, and further adjust parameter values. This program must be completed successfully before moving onto the next step.

*runValidOrchestrator.py:*

This is the third and final main program in the calibration workflow. The only mandatory argument is the unique job ID value associated with the calibration experiment. This program manages running the model with the final calibrated parameters over a specified evaluation period for the evaluation of the parameters.

*getJobID.py:*

This program will return the job ID to you in the cases where you have forgotten your unique job ID for the calibration experiment. The calibration config file is used as input into this program.
spinup.py: This is the first program that is run to initialize the calibration experiment. The only mandatory argument to this program is the unique job ID for the calibration experiment. The main purpose of this program is to run the NWM/WRF-Hydro spinup for all domains being calibrated. This program needs to successfully complete before moving onto the next step.

# Pre Calibration Step 1: Configure Setup File: "setup.parm"
The primary file you will be editing in preparation for setting up a calibration workflow job is the ‘setup.parm’ file. It is best to think of this file as a master configuration file to guide the workflow. This file contains multiple options that define how the workflow will submit jobs for models/analysis, which basins to calibrate from the database, methods for reporting errors to the user, model physics options, and paths to general parameter files and executables.
The ‘setup.parm’ file is divided up into sections: logistics, gageInfo, lsmPhysics, forcing, modelTime, hydroIO, and hydroPhysics. The first section of the setup.parm file is ‘logistics’, which guides the workflow. Let s take a look at the this file and its content: 

In [4]:
%%bash
cd /home/docker/PyWrfHydroCalib/setup_files/
ls

calib_params.tbl
domainMetaTemplate.csv
gage_list_template.csv
sens_params.tbl
setup.parm


note file: 'setup.parm' 
(Users can open a terminal session in jupyter notebook and copy/paste the following command to view this file-->
*vi setup.parm*


# Description of the setup file content. 
As mentioned above the setup file had different sections which each target a specific part of the model calibration. Below we are providing a short description of each section and its arguments. 

## logistics section
|Argument|Required|Description|
|-|-|-|
| **outDir** | Req. |Where do you want your calibration experiment to be constructed? |
|**expName** | Req. |What is the name of your experiment. This can be anything you want. |
|**acctKey** | Opt. |If you are running on a que system that requires credentials to submit a job, specify your account key here. |
|**optQueName** | Opt. |If you need to direct model simulations/jobs to a specific que, you can specify that here. |
|**nCoresModel** | Req. |How many CPUs are you running your model simulations over? |
|**nNodesModel** | Req.| If you are running across multiple nodes on a que system, specify here.| 
|**nCoresPerNode** | Req. |If you are running on a que system, specify how many CPU cores you have available per node. |
|**runSens** | Req.| Are we running sensitivity analysis? 0 - No, 1 - Yes |
|**sensParmTbl** | Req.| Path to the table of parameter values to use in sensitivity analysis. Same format as the calibration parameter table. |
|**runCalib** | Req. |Are we running calibration? 0 - No, 1 - Yes |
|**calibParmTbl** | Req.| Path to the table of parameter values to use in calibration. |
|**dailyStats** | Req. | Flag to direct worfklow to calculate error statistics on a daily scale, instead of hourly by default. Specify 1 to activate. |
|**dbBackup** |?| >> *AREZOO* Flag to turn on/off database backup. If on, the databse will be locked and backed up once an hour during execution to the job directory output file.|
|**coldStart** | Req. |Flag to direct calibration workflow to cold start your model simulation for each iteration during calibration. Specify 1 to activate. |
|**optSpinFlag** | Req. | Flag to direct the workflow to use an alternative spinup file already in place in the input directory for the basins being calibrated. This allows the user to bypass the spinup step. Specify 1 to activate. |
|**stripCalibOutputs** | Req. |If you desire to ommit outputs during an intial window for each model iteration, you can specify 1 here to activate this feature. This was designed to minimize I/O model burdens. |
|**stripCalibHours** | Req. |Specify an initial window in hours to strip outputs. This is only used if stripCalibOutputs has been activated. |
|**jobRunType** | Req. |This will specify how the model simulations and calibration code being executed. |
|**mpiCmd** | Req. |What is the MPI command being used to execute the model simulations. This is required, as the MPI command is also used in job scheduler scripts. |
|**cpuPinCmd** |Opt. |If you are running on a job scheduler, how do you want to pin specific model simulations on a compute node? This put in to allow multiple basins on one node. |
|**numIter** | Req. |How many model iterations would you like to run your calibration experiment over? |
|**calibMethod** | Req.| Right now only DDS is allowed. Future upgrades will incorporate additional calibration methods.|
|**enableStreamflowCalib**| Req.| *AREZOO* specify if streamflow will be calibrated, 0- not calibrating, 1 - calibrating | 
|**enableSnowCalib**| Req. | *AREZOO* specify if snow input data will be used in calibration, 0- not calibrating, 1 -calibrating.
|**streamflowObjectiveFunction** | Req.|Select what error metric for streamflow to minimize during the calibraion experiment. |
|**snowObjectiveFunction** | Req.|Select what error metric for snow to minimize during the calibraion experiment. |
|**soilMoistureObjectiveFunction** |Req.|Select what error metric for soil moisture to minimize during the calibraion experiment. |
|**streamflowWeight**| Req. | *AREZOO* Apply weight to streamflow calibration|
|**snowWeight**| |Req.|*AREZOO* Apply weight to the snow measurement calibration|
|**soilMoistureWeight** |Req.|*AREZOO* Apply weight to soil moisture calibration|
|**basinType**| Req. |*AREZOO* 0 - snowy, 1- slow, 2-flashy|
|**weight1Event**| Req. | *AREZOO* weights for peak bias and volume bias to get combined|
|**weight2Event**|Req.  | *AREZOO* weights for peak bias and volume bias to get combined|
|**ddsR** | Req.| This is a DDS-specific parameter that tunes how random values are generated for each iteration. |
|**enableMask**|Req.| *AREZOO* specify 1 to use mask covering part of basin, or 0 if not|
|**enableMultiSites**|Req.| *AREZOO* Specify to calibrate to 1 or more streamgages|
|**email** | Req. |Where do you want status and error messages to be directed to?|
|**wrfExe** | Req.| Path to the WRF-Hydro executable to be used in the workflow. ]
|**genParmTbl** | Req. |Path to the GENPARM.TBL file used by the model. |
|**mpParmTbl** | Req. |Path to the MPARM.TBL file used by the model. |
|**urbParmTbl** | Req. |Path to the URBPARM.TBL file used by the model. |
|**vegParmTbl** | Req. |Path to the VEGPARM.TBL file used by the model. |
|**soilParmTbl** | Req. |Path to the SOILPARM.TBL file used by the model. |
|**bSpinDate** | Opt. |Beginning date for the spinup. |
|**eSpinDate** | Opt. |Ending date for the spinup. |
|**bCalibDate** | Req. |Beginning date for each calibration iteration. |
|**eCalibDate** | Req. |Ending date for each calibration iteration. |
|**bCalibEvalDate** | Req.| The date within each calibration iteration to begin analysis. |
|**bValidDate** | Opt. |Beginning date for the validation simulation. |
|**eValidDate** | Opt. |Ending date for the validation simulation. |
|**bValidEvalDate** | Opt. |The date within the validation simulation to begin analysis. |

### Sensitivity
|Argument|Required|Description|
|-|-|-|
|**sensParmSample** | Opt. |Sensitivity parameter sample size|
|**sensBatchNum** | Opt. | How many sensitivity simulations to run at once. |
|**bSensDate** | Opt. |Beginning date for the sensitivity model simulation. |
|**eSensDate** | Opt. |Ending date for the sensitivity model simulation. |
|**bSensEvalDate** | Opt. |The date within the sensitivity simulation to begin analysis. |

### gageInfo
|Argument|Required|Description|
|-|-|-|
|**gageListSQL** | Req. |SQL command to extract basins for calibration out of the database file. |
|**gageListFile** | Opt. |Alternative list of basins to calibrate instead of using an SQL command. |

### lsmPhysics
|Argument|Required|Description|
|-|-|-|
|**SplitOutputCount**| Req.| Output Options: 1 - output 1 file per output time step, 0 - append all the timesteps to one file called LDASOUT_DOMAIN1.nc |
|**dynVegOption** | Req. |DYNAMIC_VEG_OPTION for NoahMP. |
|**canStomResOption** | Req. |CANOPY_STOMATAL_RESISTANCE_OPTION for NoahMP. |
|**btrOption** | Req. BTR_OPTION for NoahMP. |
|**runoffOption** | Req.| RUNOFF_OPTION for NoahMP. |
|**sfcDragOption** | Req. |SURFACE_DRAG_OPTINO for NoahMP. |
|**frzSoilOption** | Req. |FROZEN_SOIL_OPTION for NoahMP. |
|**supCoolOption** | Req. |SUPERCOOLED_WATER_OPTION for NoahMP. |
|**radTransferOption** | Req. |RADIATIVE_TRANSFER_OPTION for NoahMP. |
|**snAlbOption** | Req.| SNOW_ALBEDO_OPTION for NoahMP. |
|**pcpPartOption** | Req. |PCP_PARTITION_OPTION. |
|**tbotOption** | Req. |TBOT_OPTINO for NoahMP. |
|**tempTimeScOption** | Req.| TEMP_TIME_SCHEME_OPTION for NoahMP. |
|**sfcResOption** | Req. |SURFACE_RESISTANCE_OPTION for NoahMP. |
|**glacierOption** | Req. |GLACIER_OPTION for NoahMP. |
|**soilThick** | Req. |Soil thicknesses for specified soil layers in NoahMP. |
|**zLvl** | Req.| Level of wind speeds in NoahMP. |

### forcing
|Argument|Required|Description|
|-|-|-|
|**forceType** | Req. |Specified forcing type. |

### modelTime
|Argument|Required|Description|
|-|-|-|
|**forceDt** | Req. |Input forcing timestep in seconds. |
|**lsmDt** | Req. |NoahMP timestep in seconds. |
|**lsmOutDt** | Req. |NoahMp output timestep in seconds. |
|**lsmRstFreq** | Req. |NoahMp restart frequency in seconds. |
|**hydroRstFreq** | Req. |WRF-Hydro restart frequency in seconds. |
|**hydroOutDt**  |Req. |WRF-Hydro output frequency in seconds. |

### hydroIO
|Argument|Required|Description|
|-|-|-|
|**rstType** | Req. |Flag for overwritting accumulation vars in restart file. |
|**SplitOoutputCount**| Req.| 1- output one CHANOB file per timestep, 0 - output one file containing all timesteps: CHANOBS_DOMAIN1.nc|
|**ioConfigOutputs** | Req. |Output flag for varible grouping in WRF-Hydro. |
|**ioFormOutputs** | Req. |Flag to for specifying output format. |
|**chrtoutDomain** | Req. |Flag to turn on CHRTOUT_DOMAIN files. |
|**chanObsDomain** | Req. |*AREZOO* Flag to turn on CHANOBS_DOMAIN files. |
|**chrtoutGrid** | Req. |Flag to turn on CHRTOUT_GRID files. |
|**lsmDomain** | Req. |Flag to turn on LSMOUT_DOMAIN files. |
|**rtoutDOmain** | Req.| Flag to turn on RTOUT_DOMAIN files. |
|**gwOut** | Req. Flag to turn on GWOUT_DOMAIN files. |
|**lakeOut** | Req. |Flag to turn on LAKEOUT_DOMAIN files. |
|**frxstOut** | Req. |Flag to turn on FRXST output text files. |
|**resetHydroAcc** | Req. |Flag to reset accumulation variables in the restart files. |
|**streamOrderOut** | Req.| Flag to specify the minimum Strahler order to output. |

### hydroPhysics
|Argument|Required|Description|
|-|-|-|
|**dtChSec** | Req.| Channel routing timesetp in seconds. |
|**dtTerSec** | Req. |Surface and subsurface routing timestep in seconds. |
|**subRouting** | Req.| Flag to turn on/off subsurface routing. |
|**ovrRouting** | Req. | Flag to turn on/off overland flow routing. |
|**channelRouting** | Req. | Flag to turn on/off channel routing. |
|**rtOpt** | Req. | Overland/subsurface routing option. |
|**chanRtOpt** | Req. |Channel routing option. |
|**udmpOpt** | Req. | User-defined spatial mapping flag to turn on/off. |
|**gwBaseSw** | Req. | Groundwater option. |
|**gwRestart** | Req. | Flag to use restart states in groundwater scheme. |
|**enableCompoundChannel** | Req. | Flag to activate compound channel in the hydro.namelist. 1 - on, 0 - off.|
|**compoundChannel** | Req. |Activation flag for compound channel. enableCompoundChannel must be on. |
|**enableGwBucketLoss** | Req. |Flag to activate groundwater bucket loss in hydro.namelist. 1 - on, 0 - off. |
|**bucket_loss** | Req.| Activation flag for groundwater bucket loss. enableGwBucketLoss must be on. |

## Pre Calibration Step 2: Configure Calibration Parameter Selection "calib_parms.tbl"
In addition to the ‘setup.parm’ file, the ‘calib_parms.tbl’ file is needed to direct the workflow to determine which model parameters will be calibrated, along with the range of parameter values. A template table is located under /setup_files/calib_parms.tbl which you can copy and edit for your own calibration workflow experiment.


In [ ]:
%%bash
cd /home/docker/PyWrfHydroCalib/setup_files
cat calib_params.tbl

parameter,calib_flag,minValue,maxValue,ini
bexp,          1,      0.40,      1.90,      1.0
smcmax,        1,      0.80,      1.20,      1.0
dksat,         1,      0.20,      10.0,      1.0
refkdt,        1,      0.10,      4.00,      0.6
slope,         1,      0.00,      1.00,      0.1
retdeprtfac,   1,      0.10,      10.0,      1.0
lksatfac,      1,      10.0,      10000.0,   1000.0
zmax,          1,      10.0,      250.0,     25.0
expon,         1,      1.00,      8.0,       1.75
Coeff,         1,      0.0001,    0.1,       0.001
cwpvt,         1,      0.50,      2.0,       1.0
vcmx25,        1,      0.60,      1.4,       1.0
mp,            1,      0.60,      1.4,       1.0
hvt,           1,      0.25,      1.5,       1.0
mfsno,         1,      0.50,      2.0,       1.0
rsurfexp,      0,      1.0,       6.0,       5.0
Bw,            0,      0.1,       10.0,      1.0
HLINK,         0,      0.1,       10.0,      1.0
ChSSlp,        0,      0.1,       10.0,      1.0
MannN,         0,  

*AREZOO*
Within this table, you will find:

- Parameter -- all the potential parameters to be calibrated
- calib_flag -- of 1 or 0. This flag will turn calibration on (1) for that parameter or off (0). 
- minValues -- The minimum range value for parameter calibration
- maxValues -- The maximum range value for parameter calibration
- ini -- column specifies the default values to be used for either default un-calibrated values, or the initial values going into the calibration workflow. It is up to you to determine what range is best for your calibration experiment. It is highly encouraged to perform a sensitivity analysis over your region of interest to help determine which parameters have significant impact on hydrologic response.


## Conclusion:

Once the setup.parm file and calibration parameters are set and verified, we can begin the process of initializing databases for calibration. Proceed to *Lesson 2 - Create Database*